In [1]:
import pickle
import pandas as pd
import numpy as np
import optuna

#     optuna_ex1_hyperparameters_per_dataset.py
#  -> analysis_ex1_hyperparameters.ipynb
#  -> benchmark_ex1_best_hyperparameters.py
#  -> analysis_ex1_hyperparameters_best.ipynb

In [2]:
storage = "sqlite:///optuna_databases/optuna_ex1_parameter_tuning_2D_e3_d2.sqlite3" #CHANGE


In [3]:
studies = optuna.study.get_all_study_summaries(storage=storage)

In [4]:
loaded_study = optuna.load_study(study_name=studies[0].study_name, storage=storage)
df = loaded_study.trials_dataframe()
for key, value in loaded_study.system_attrs.items():
    df[key] = value
for study in studies[1:]:
    loaded_study = optuna.load_study(study_name=study.study_name , storage=storage)
    df_new = loaded_study.trials_dataframe()
    for key, value in loaded_study.system_attrs.items():
        df_new[key] = value
    df = pd.concat([df, df_new])
df.reset_index(inplace=True, drop=True)

In [5]:
df.columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_init_learning_rate', 'params_learning_rate_decay',
       'params_max_depth', 'params_n_experts', 'params_optimization_method',
       'params_use_2_dim_clustering', 'params_use_2_dim_gate_based_on',
       'user_attrs_accuracy_train', 'user_attrs_accuracy_val',
       'user_attrs_std_train', 'user_attrs_std_val', 'state', 'Data X',
       'Data y', 'initialization_method', 'params_alpha', 'params_beta',
       'params_gamma', 'params_mean_precision_prior',
       'params_weight_concentration_prior',
       'params_weight_concentration_prior_type', 'params_weight_cutoff'],
      dtype='object')

In [6]:
df.head(1)

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,Data X,Data y,initialization_method,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
0,0,0.71582,2021-07-29 17:26:54.287672,2021-07-29 17:26:59.056671,0 days 00:00:04.768999,8.806749,0.987118,2,3,ridge_regression,...,abalone_input.pd,abalone_target.pd,str,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.initialization_method.value_counts()

Kmeans_init      1212
str              1212
KDTmeans_init    1212
BGM_init         1212
Name: initialization_method, dtype: int64

In [8]:
np.unique(df.initialization_method)

array(['BGM_init', 'KDTmeans_init', 'Kmeans_init', 'str'], dtype=object)

In [9]:
np.unique(df["Data X"])

array(['abalone_input.pd', 'adult_input.pd', 'bank_input.pd',
       'banknote_input.pd', 'breast_cancer_input.np', 'cars_input.pd',
       'contraceptive_input.pd', 'generated6_input.np', 'hrss_input.pd',
       'iris_input.pd', 'steel_input.pd', 'students_input.pd'],
      dtype=object)

In [21]:
top = 10
df_best = pd.DataFrame()
for dataset in np.unique(df["Data X"]):
    df_new = df[df["Data X"] == dataset].sort_values(by=["user_attrs_accuracy_val"], ascending=False).head(top)
    df_best = pd.concat([df_best, df_new])
df_best.reset_index(inplace=True, drop=True)


In [22]:
df_best.shape

(120, 27)

In [23]:
df_best.head(2)

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,Data X,Data y,initialization_method,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
0,48,0.737373,2021-07-29 19:32:44.315334,2021-07-29 19:32:54.882780,0 days 00:00:10.567446,137.423977,0.996847,2,3,ridge_regression,...,abalone_input.pd,abalone_target.pd,BGM_init,NaN,NaN,NaN,0.349378,0.269386,dirichlet_distribution,0.0
1,88,0.733782,2021-07-29 19:39:48.829076,2021-07-29 19:39:59.649793,0 days 00:00:10.820717,127.137474,0.998689,2,3,ridge_regression,...,abalone_input.pd,abalone_target.pd,BGM_init,NaN,NaN,NaN,0.671007,0.290044,dirichlet_distribution,0.0


In [24]:
#pickle.dump(df_best, open("dataframes/ex1_df_top10_hyperparameters_per_dataset_FD_e3_d2.pd", "wb"))
pickle.dump(df_best, open("dataframes/ex1_df_top10_hyperparameters_per_dataset_2D_e3_d2.pd", "wb")) #CHANGE

In [25]:
df.groupby(["Data X"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
Data X,,,,,,,,,,,,,,,,,
abalone_input.pd,49.009901,0.709470,99.809565,0.989751,2.0,3.0,True,0.725937,0.709470,0.006648,0.011873,1.944604,0.378032,0.200535,0.577498,0.404890,0.0
adult_input.pd,49.009901,0.834996,65.701799,0.990354,2.0,3.0,True,0.837549,0.834996,0.002054,0.006584,2.391523,0.159688,0.331515,0.508724,0.599633,0.0
bank_input.pd,49.009901,0.906775,68.485053,0.986449,2.0,3.0,True,0.909759,0.906775,0.001318,0.004691,3.107680,0.204523,0.288042,0.687174,0.466602,0.0
banknote_input.pd,49.009901,0.977458,81.340996,0.988332,2.0,3.0,True,0.987118,0.977458,0.003854,0.007690,1.612292,0.206658,0.143187,0.770314,0.611813,0.0
breast_cancer_input.np,49.009901,0.922598,61.284745,0.987819,2.0,3.0,True,0.964526,0.922598,0.005939,0.018299,2.312267,0.198241,0.221582,0.696980,0.348388,0.0
cars_input.pd,49.009901,0.795082,82.761344,0.985129,2.0,3.0,True,0.804452,0.795082,0.009717,0.015968,2.024554,0.161061,0.185896,0.464478,0.593035,0.0
contraceptive_input.pd,49.009901,0.539778,96.026573,0.986152,2.0,3.0,True,0.569172,0.539778,0.011478,0.020747,2.967495,0.367225,0.210613,0.651171,0.473157,0.0
generated6_input.np,49.009901,0.926108,96.712580,0.987859,2.0,3.0,True,0.931598,0.926108,0.011420,0.013798,2.701032,0.164902,0.112983,0.325528,0.693757,0.0
hrss_input.pd,49.009901,0.771224,75.354605,0.990805,2.0,3.0,True,0.772132,0.771224,0.002261,0.004730,2.982472,0.289240,0.185007,0.522882,0.466111,0.0


In [26]:
df_best.groupby(["Data X"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
Data X,,,,,,,,,,,,,,,,,
abalone_input.pd,60.8,0.731674,126.973087,0.993876,2.0,3.0,True,0.740667,0.731674,0.006462,0.009714,NaN,NaN,NaN,0.537324,0.380489,0.0
adult_input.pd,52.0,0.840117,26.619247,0.996597,2.0,3.0,True,0.840273,0.840117,0.001037,0.002947,2.284642,0.144427,0.349933,NaN,NaN,NaN
bank_input.pd,52.7,0.911695,44.431908,0.989367,2.0,3.0,True,0.912744,0.911695,0.001575,0.003204,NaN,NaN,NaN,0.749154,0.393702,0.0
banknote_input.pd,70.5,0.995190,117.299542,0.981946,2.0,3.0,True,0.997473,0.995190,0.001279,0.003651,NaN,NaN,NaN,0.966254,0.664669,0.0
breast_cancer_input.np,50.6,0.941482,48.302150,0.987416,2.0,3.0,True,0.967193,0.941482,0.004973,0.012726,2.368202,0.092829,0.247154,0.757227,0.244417,0.0
cars_input.pd,61.9,0.810069,79.100372,0.982035,2.0,3.0,True,0.812288,0.810069,0.008584,0.015956,2.287774,0.116566,0.163702,0.419682,0.993891,0.0
contraceptive_input.pd,54.1,0.575288,119.966788,0.981234,2.0,3.0,True,0.589183,0.575288,0.009641,0.020559,NaN,NaN,NaN,NaN,NaN,NaN
generated6_input.np,62.3,0.968680,118.853659,0.983027,2.0,3.0,True,0.971813,0.968680,0.004159,0.006115,2.702514,0.098295,0.070032,0.186915,0.822405,0.0
hrss_input.pd,52.1,0.775453,82.479360,0.993243,2.0,3.0,True,0.774935,0.775453,0.001873,0.005635,3.199291,0.302812,0.139829,NaN,NaN,NaN


In [27]:
df["params_use_2_dim_gate_based_on"].value_counts(dropna=False)

feature_importance                 694
feature_importance_lr_max          677
feature_importance_lr              636
feature_importance_lda_max         635
feature_importance_lda             587
feature_importance_xgb             586
feature_importance_pca_loadings    532
PCA                                501
Name: params_use_2_dim_gate_based_on, dtype: int64

In [35]:
df_best["params_optimization_method"].value_counts()

least_squares_linear_regression    72
ridge_regression                   42
lasso_regression                    6
Name: params_optimization_method, dtype: int64

In [40]:
df_best.groupby(["params_optimization_method"]).median()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
params_optimization_method,,,,,,,,,,,,,,,,,
lasso_regression,74.5,0.518780,73.197445,0.985656,2.0,3.0,True,0.556301,0.518780,0.010482,0.016121,NaN,NaN,NaN,NaN,NaN,NaN
least_squares_linear_regression,55.5,0.811632,110.572272,0.985380,2.0,3.0,True,0.814525,0.811632,0.003864,0.008965,2.487167,0.057464,0.101439,0.738174,0.312895,0.0
ridge_regression,53.0,0.840279,80.497145,0.990411,2.0,3.0,True,0.840439,0.840279,0.005513,0.008856,2.928541,0.322649,0.206478,0.635707,0.397779,0.0


In [37]:
df_best["params_use_2_dim_gate_based_on"].value_counts()

feature_importance_xgb             19
feature_importance_lda_max         16
feature_importance_lr_max          16
feature_importance_pca_loadings    15
feature_importance_lr              15
feature_importance_lda             14
PCA                                14
feature_importance                 11
Name: params_use_2_dim_gate_based_on, dtype: int64